<a href="https://colab.research.google.com/github/elizabethyoo/team_9/blob/EDA_linegraphs/OLS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [88]:
import pandas as pd
import glob
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
from matplotlib import animation as animation
import matplotlib.dates as mdates

from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
# from regressors import stats   


pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [89]:
df = pd.read_csv('/content/drive/MyDrive/group_9/Datasets/hps_sbps_unemp_vac.csv', index_col='Date')
df_no_peak = df[(df.index<'2020-10-31') | (df.index >'2021-02-28')] 

In [90]:
col_to_drop = ['year', 'month','tot_cases', 'tot_death','population_size','total_deaths_per_1_mill_pop',
       'total_cases_per_1_mill_pop','Administered_Dose1_Pop_Pct', 'Administered_Dose1_Recip_12PlusPop_Pct',
       'Administered_Dose1_Recip_18PlusPop_Pct',
       'Administered_Dose1_Recip_65PlusPop_Pct', 
       'Series_Complete_12PlusPop_Pct', 'Series_Complete_18PlusPop_Pct',
       'Series_Complete_65PlusPop_Pct', 'Additional_Doses_Vax_Pct',
       'Additional_Doses_18Plus_Vax_Pct', 'Additional_Doses_50Plus_Vax_Pct',
       'Additional_Doses_65Plus_Vax_Pct','female','state',
       'childcare_issue', 'financial_stress', 'market_challenges',
       'markets_challenges','operational_challenges','new_case', 'new_death',]


df_to_model = df.drop(columns=col_to_drop)  
df_to_model['Series_Complete_Pop_Pct'] = df_to_model['Series_Complete_Pop_Pct'].fillna(0)

In [91]:
df_to_model.head(2)

,age_18_24,age_25_39,age_40,age_50,age_60,age_70_up,child,education_bachelor,education_hs,education_less_hs,education_associate,income_100K,income_150K,income_200K,income_25K,income_35K,income_50K,income_75K,income_less_25K,race_other,race_asian,race_black,race_white,food_scarcity,housing_insecurity,male,hispanic_origin,expected_recovery,overall_sentiment,unemployment_rate,new_cases_per_1_mill_pop,new_deaths_per_1_mill_pop,fatality_rate,Series_Complete_Pop_Pct
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-04-30,9.7022,34.3529,14.8741,16.0768,15.0907,9.9033,0.0000,27.5564,36.0829,1.2940,35.0667,15.1546,5.9602,6.5873,9.6292,9.7133,17.4755,14.2922,8.2719,22.1747,6.9447,2.8202,68.0603,7.4137,1.9379,51.1094,6.4097,-0.5900,-0.6600,12.4000,310.4664,4.1031,3.0277,0.0000
2020-05-31,9.7347,32.9765,15.5323,16.5215,14.8570,9.7349,0.6431,27.5405,36.2782,1.1069,35.0744,13.8033,7.9779,5.6725,7.4605,8.4605,17.6799,12.3827,13.4845,24.0058,5.1436,2.1758,68.6748,6.9246,2.5193,51.1094,6.2471,-0.5940,-0.6460,12.0000,147.7109,5.4708,2.6981,0.0000


In [92]:
X = df_to_model.drop(columns='expected_recovery')
y = df_to_model.loc[:,'expected_recovery']

scaler = MinMaxScaler()
X_transformed = scaler.fit_transform(X)

# Sklearn

In [94]:
model = Ridge()
model.fit(X_transformed, y)
coef = pd.DataFrame(zip(X.columns,model.coef_ ), columns = ['Features', 'Coef'])

# Coef didn't have any p-value

#Statmodel

In [96]:
X_transformed_sm = pd.DataFrame(X_transformed, columns= X.columns)
X_transformed_sm.set_index(y.index, inplace=True)

X_transformed_sm = sm.add_constant(X_transformed_sm)
model = sm.OLS(y, X_transformed_sm)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:      expected_recovery   R-squared:                       0.874
Model:                            OLS   Adj. R-squared:                  0.869
Method:                 Least Squares   F-statistic:                     161.1
Date:                Sun, 24 Oct 2021   Prob (F-statistic):          7.78e-319
Time:                        16:42:40   Log-Likelihood:                 1826.4
No. Observations:                 800   AIC:                            -3585.
Df Residuals:                     766   BIC:                            -3425.
Df Model:                          33                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

In [104]:
X_transformed_sm = sm.add_constant(X_transformed_sm)
model = sm.OLS(y, X_transformed_sm)
results_reg = model.fit_regularized(method='sqrt_lasso', alpha=0.2, L1_wt=0.0)
print(results_reg.summary())

NotImplementedError: ignored

In [111]:
results_reg.params.

SyntaxError: ignored

# Withheld peak cases data period


In [109]:
df_to_model = df_no_peak.drop(columns=col_to_drop)  
df_to_model['Series_Complete_Pop_Pct'] = df_to_model['Series_Complete_Pop_Pct'].fillna(0)

X = df_to_model.drop(columns='expected_recovery')
y = df_to_model.loc[:,'expected_recovery']

scaler = MinMaxScaler()
X_transformed = scaler.fit_transform(X)

In [110]:
X_transformed_sm = pd.DataFrame(X_transformed, columns= X.columns)
X_transformed_sm.set_index(y.index, inplace=True)

X_transformed_sm = sm.add_constant(X_transformed_sm)
model = sm.OLS(y, X_transformed_sm)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:      expected_recovery   R-squared:                       0.892
Model:                            OLS   Adj. R-squared:                  0.885
Method:                 Least Squares   F-statistic:                     128.8
Date:                Sun, 24 Oct 2021   Prob (F-statistic):          1.02e-225
Time:                        16:50:36   Log-Likelihood:                 1253.4
No. Observations:                 550   AIC:                            -2439.
Df Residuals:                     516   BIC:                            -2292.
Df Model:                          33                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

In [117]:
r = results.get_robustcov_results()